In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langmem import create_memory_manager
from pydantic import BaseModel
from langchain_openai.chat_models import ChatOpenAI



llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


class Person(BaseModel):
    """Store a person's name, role, and preferences."""
    name: str
    role: str
    preferences: list[str] | None = None


manager = create_memory_manager(
    llm,
    schemas=[Person],
    instructions="Extract people's names, roles, and any mentioned preferences.",
    enable_inserts=True,
    enable_deletes=True,
)

In [ ]:
conversation = [
    {
        "role": "user",
        "content": (
            "John is a senior developer who loves coffee. "
            "Alice is a junior developer who hates coffee."
        )
    }
]
memories = manager.invoke({"messages": conversation})
print(memories)

In [ ]:
conversation_no_extraction = [
    {
        "role": "user",
        "content": (
            "Today it rained for two hours, and then the sun came out."
        )
    }
]

memories_no_extraction = manager.invoke({"messages": conversation_no_extraction})
print(memories_no_extraction)

In [ ]:
from langgraph.store.memory import InMemoryStore

from langmem import create_manage_memory_tool, create_search_memory_tool


store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small",
    }
)

In [ ]:
tools=[
    create_manage_memory_tool(namespace=("memories", "abc"), store=store),
    create_search_memory_tool(namespace=("memories", "abc"), store=store),
]

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools.invoke(input="hi")

In [ ]:
llm_with_tools.invoke(input="what do you know about me?")

In [ ]:
llm_with_tools.invoke(input="I love spaghetti")

In [ ]:
llm_with_tools.invoke(input="what do you know about me?")